In [1]:
'''
Import the dataset

Perform NLP preprocessing

Apply ML models

Training/Testing/Hyperparameter tuning

'''

'\nImport the dataset\n\nPerform NLP preprocessing\n\nApply ML models\n\nTraining/Testing/Hyperparameter tuning\n\n'

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.svm import LinearSVC

from sklearn.metrics import precision_recall_fscore_support

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('dataset/input_train.csv')

#add column title
#add tsne later
df.columns = ['document_label', 'document_text']
df = df[pd.notnull(df['document_text'])]

df.shape
X = df['document_text']
y = df['document_label']

X.describe()

count                                                 61709
unique                                                59753
top       bf064c332aa1 079935e500e5 1a4dd36c6de0 7efa289...
freq                                                     11
Name: document_text, dtype: object

In [4]:
#research more on this. 
# vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 2), min_df = 1, max_df = 0.9, max_features = 5000)

# vector_content = vectorizer.fit_transform(X.apply(lambda x: np.str_(x)))

# print(vector_content.shape)

vectorizer = CountVectorizer(ngram_range=(1,2), max_features=1000)
vector_content = vectorizer.fit_transform(X.apply(lambda x: np.str_(x)))
print(vector_content.shape)

(61709, 1000)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    vector_content, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=1)

In [6]:
# Handle null document_text columns
# count_nan = len(df) - df.count()
#print(df['document_text'].isna().sum())

In [7]:
#import model
svm_clf = SVC(gamma="scale")
svm_clf.fit(X_train[:30000], y_train[:30000])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
df_test = pd.read_csv('dataset/input_test_.csv')

print(df_test.shape)
#add column title
#add tsne later
df_test.columns = ['document_label', 'document_text']
df_test = df_test[pd.notnull(df_test['document_text'])]

df.shape
X_t = df_test['document_text']
y_t = df_test['document_label']

df_test.head()

df_test.to_csv(r'dataset/input_test_label.csv', index = False, encoding='utf-8')

(172, 2)


In [9]:
df_label = pd.read_csv('dataset/input_test_label.csv')
X_ = df_label['document_text']

print(X_.shape)
vectorize = CountVectorizer(ngram_range=(1,2), max_features=1000)
vec_content = vectorize.fit_transform(X_.apply(lambda x: np.str_(x)))
print(vec_content.shape)
                                      
y_t = df_label['document_label'].values.tolist()                                        
y_label = svm_clf.predict(vec_content)   

print(precision_recall_fscore_support(y_t, y_label, average='macro'))     

(172,)
(172, 1000)
(0.12105263157894737, 0.13355973528387322, 0.08289246693502014, None)


C:\Python27\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
y_pred = svm_clf.predict(X_val)

print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_val, y_pred, 
                                    target_names= df['document_label'].unique()))


				CLASSIFICATIION METRICS

                         precision    recall  f1-score   support

         RETURNED CHECK       0.81      0.29      0.43        45
                   BILL       0.90      0.89      0.90      3706
          POLICY CHANGE       0.40      0.04      0.07        55
    CANCELLATION NOTICE       0.84      0.86      0.85      1780
            DECLARATION       0.80      0.87      0.83      1929
     CHANGE ENDORSEMENT       0.74      0.79      0.77       178
     NON-RENEWAL NOTICE       0.57      0.18      0.28       193
                 BINDER       0.90      0.89      0.89       972
   REINSTATEMENT NOTICE       0.94      0.59      0.72       150
   DELETION OF INTEREST       1.00      0.29      0.45        41
      EXPIRATION NOTICE       0.94      0.70      0.80       113
INTENT TO CANCEL NOTICE       0.78      0.86      0.82      2119
            APPLICATION       0.92      0.91      0.92       896
            BILL BINDER       0.98      0.75      0.85      

In [11]:
print(precision_recall_fscore_support(y_val, y_pred, average='macro'))
y_pred_test = svm_clf.predict(X_test)
print(precision_recall_fscore_support(y_test, y_pred_test, average='macro'))

(0.8232001016104897, 0.6355821553463901, 0.6830182780049482, None)
(0.8224502012196832, 0.6332329502983584, 0.67518174611608, None)


In [12]:
#multinomialNB
NB_classifier = MultinomialNB().fit(X_train, y_train)

y_pred_val = NB_classifier.predict(X_val)
y_pred_test_NB = NB_classifier.predict(X_test)


print("nb validation")
print(precision_recall_fscore_support(y_val, y_pred_val, average='macro'))



nb validation
(0.48614410981510753, 0.7006008662940548, 0.5179960498635788, None)


In [13]:
#sgd classifier

from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier(max_iter=1000, tol=1e-3)

sgd_model.fit(X_train, y_train)

y_pred_val_ = sgd_model.predict(X_val)
y_pred_test_sgd = sgd_model.predict(X_test)

print("sgd validation")
print(precision_recall_fscore_support(y_val, y_pred_val_, average='macro'))

print("sgd test")
print(precision_recall_fscore_support(y_test, y_pred_test_sgd, average='macro'))

                                      
y_lab = sgd_model.predict(vec_content)
print(precision_recall_fscore_support(y_t, y_lab, average='macro'))



sgd validation
(0.7633549603927785, 0.6865677312503299, 0.716039894788853, None)
sgd test
(0.760640696200532, 0.6814352232199178, 0.7093594917140947, None)
(0.08339108339108338, 0.07083284956848175, 0.04370198371746359, None)


C:\Python27\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python27\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [23]:

#confusion matrix

from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt     

ax= plt.subplot()

labels = df["document_label"].unique()

cm = confusion_matrix(y_test, y_pred_test_sgd, labels)
print(cm)

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

[[ 119    8   12    0    0    0    0    2    0    0    0    0    0    0]
 [   1 3199  176  214   25    2    1   49    6    4   14    1    0   13]
 [   8   80 1836   45   10   20    4   55   14   20    3    1    4    6]
 [   1   66   55 1725    5    1    2    5   34   70    4    2    0    0]
 [   2   27   50    7   46    2    0   45    0   10    1    0    1    0]
 [   0    1   34    3    1  136    0    0    0    1    1    0    0    0]
 [   0    0    6   13    0    1  126    0    0    5    0    1    0    0]
 [   2   42  215   14   46    5    1 1468    2    5    1    0    2    0]
 [   2   11   22   33    2    0    0    0  792   10    2    0    0    0]
 [   0    1    9   83    0    3    1    2    3  830    0    0    0    0]
 [   0   26    3    6    1    1    1    1    0    0  101    0    0    0]
 [   0    6    1   18    0    0    0    0    1    3    0   16    0    0]
 [   0    2   12    0    2    0    0    9    0    0    0    0   20    0]
 [   0   28   16    2    1    1    0    5    0    1

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 2 Axes>

In [15]:
models = [
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0)
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, vector_content, y, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

C:\Python27\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Python27\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Python27\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Python27\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [16]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

,Mean Accuracy,Standard deviation
model_name,,
LinearSVC,0.851108,0.001258
LogisticRegression,0.860329,0.003330
MultinomialNB,0.681181,0.002016
